In [11]:
import numpy as np
from netCDF4 import Dataset
import cartopy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import wrf
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
# get data
data_dir = '/home/jcurtis2/hackathon_data/'
wrf_filename = '%straining.nc' % data_dir
ncf = Dataset(wrf_filename, "r", format="NETCDF4")

In [13]:
lats = ncf.variables['pmc_SO4'][0,0,0,0].data

In [14]:
# Input variables:

input_vars = ['TOT_NUM_CONC',
 'TOT_MASS_CONC',
 'pmc_SO4',
 'pmc_NO3',
 'pmc_Cl',
 'pmc_NH4',
 'pmc_ARO1',
 'pmc_ARO2',
 'pmc_ALK1',
 'pmc_OLE1',
 'pmc_API1',
 'pmc_API2',
 'pmc_LIM1',
 'pmc_LIM2',
 'pmc_OC',
 'pmc_BC',
 'pmc_H2O',
 'TEMPERATURE',
 'REL_HUMID',
 'ALT',
 'Z',
 'XLAT',
 'XLONG',
 'h2so4',
 'hno3',
 'hcl',
 'nh3',
 'no',
 'no2',
 'no3',
 'n2o5',
 'hono',
 'hno4',
 'o3',
 'o1d',
 'O3P',
 'oh',
 'ho2',
 'h2o2',
 'co',
 'so2',
 'ch4',
 'c2h6',
 'ch3o2',
 'ethp',
 'hcho',
 'ch3oh',
 'ANOL',
 'ch3ooh',
 'ETHOOH',
 'ald2',
 'hcooh',
 'RCOOH',
 'c2o3',
 'pan',
 'aro1',
 'aro2',
 'alk1',
 'ole1',
 'api1',
 'api2',
 'lim1',
 'lim2',
 'par',
 'AONE',
 'mgly',
 'eth',
 'OLET',
 'OLEI',
 'tol',
 'xyl',
 'cres',
 'to2',
 'cro',
 'open',
 'onit',
 'rooh',
 'ro2',
 'ano2',
 'nap',
 'xo2',
 'xpar',
 'isop',
 'isoprd',
 'isopp',
 'isopn',
 'isopo2',
 'api',
 'lim',
 'dms',
 'msa',
 'dmso',
 'dmso2',
 'ch3so2h',
 'ch3sch2oo',
 'ch3so2',
 'ch3so3',
 'ch3so2oo',
 'ch3so2ch2oo',
 'SULFHOX',
 'P',
 'PB']

out_vars = ['ccn_001',
 'ccn_003',
 'ccn_006',
 'CHI',
 'CHI_CCN',
 'D_ALPHA',
 'D_GAMMA',
 'D_ALPHA_CCN',
 'D_GAMMA_CCN',
 'PM25']


input_vars = ['isop',
 'isoprd',
 'isopp',
 'isopn',
 'isopo2']

output_vars = ['PM25', 'D_GAMMA_CCN']


In [15]:
# flatten out data

# training
data_frame_input = {}
data_frame_output = {}

# keep the last 100 data points for testing
test_input = {}
test_output = {}

for input in range(len(input_vars)):
    if(input_vars[input]== 'XLAT' or input_vars[input] == 'XLONG'):
        t = ncf.variables[input_vars[input]][:,:,:].flatten()
        
        list1 = t / np.log(t) 
        for i in range(133):
            list.append(list1)
    else:
        t = ncf.variables[input_vars[input]][:,:,:,:].flatten()
        list = t / np.log(t)
        
    test_input[input_vars[input]] = list[-100:]
    data_frame_input[input_vars[input]] = list[:-100]
    
for input in range(len(output_vars)):
    t = ncf.variables[output_vars[input]][:,:,:,:].flatten()
    list = t / np.log(t)
    test_output[output_vars[input]] = list[-100:]
    data_frame_output[output_vars[input]] = list[:-100]

/tmp/ipykernel_3302723/3374787728.py:20: RuntimeWarning: divide by zero encountered in log
  list = t / np.log(t)


In [16]:

X = pd.DataFrame(data_frame_input)
y = pd.DataFrame(data_frame_output)

X_test = pd.DataFrame(test_input)
y_test = pd.DataFrame(test_output)

X_test

,isop,isoprd,isopp,isopn,isopo2
0,-2.714552e-18,-4.199991e-16,-2.714382e-18,-2.714382e-18,-2.714419e-18
1,-2.714507e-18,-3.788814e-16,-2.714418e-18,-2.714412e-18,-2.714446e-18
2,-2.714601e-18,-4.239877e-16,-2.714524e-18,-2.714507e-18,-2.714538e-18
3,-2.714473e-18,-3.524237e-16,-2.714472e-18,-2.714476e-18,-2.714469e-18
4,-2.714402e-18,-3.025023e-16,-2.714452e-18,-2.714457e-18,-2.714439e-18
...,...,...,...,...,...
95,-2.715044e-18,-3.523457e-15,-2.714532e-18,-2.714505e-18,-2.714657e-18
96,-2.715041e-18,-3.914836e-15,-2.714508e-18,-2.714492e-18,-2.714638e-18
97,-2.715046e-18,-4.052030e-15,-2.714506e-18,-2.714493e-18,-2.714630e-18
98,-2.715160e-18,-4.442820e-15,-2.714518e-18,-2.714503e-18,-2.714630e-18


In [17]:
# with sklearn

regr = linear_model.LinearRegression() # Create linear regression object
regr.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# prediction with sklearn

print ('Predicted PM25 at some time: \n', regr.predict([X_test]))
print(y_test['PM25'][0])

isop      1.000076e-16
isoprd    1.341576e-14
isopp     1.000015e-16
isopn     1.000015e-16
isopo2    1.000028e-16
Name: 0, dtype: float32